# Mapas Coropléticos

Los mapas coropléticos, se caracterizan por usar sombreado, tintes o patrones, para mostrar las diferencias de un valor en proporción en una ubicación geográfica o región, es decir, muestran de manera rápida, las diferencias relativas a través de sombreados, que va del claro (valores menos frecuentes o inferiores) a oscuro (más frecuentes o superiores).

Los mapas coropléticos son una excelente opción en los siguientes casos:
- Para mostrar información cuantitativa en un mapa.
- Para mostrar las relaciones y patrones espaciales.
- Cuando los datos están normalizados.
- Cuando se trabaja con datos socioeconómicos.
- Cuando las regiones definidas son importantes.
- Para obtener una visión general de la distribución en las ubicaciones geográfica.

## Uso de GeoJson

Vamos a cargar un archivo GeoJSON que representa los estados de Estados Unidos.

In [ ]:
import json

import requests

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/main/examples/data"
)
us_states = f"{url}/us-states.json"

geo_json_data = json.loads(requests.get(us_states).text)

Una primera forma de dibujarlo en un mapa, es simplemente utilizar `folium.GeoJson`

In [ ]:
# Alternativa visual 1

import folium

m = folium.Map([43, -100], width='50', height='50', zoom_start=4)

folium.GeoJson(geo_json_data).add_to(m)

m

In [ ]:
# Alternativa visual 2

from branca.element import Figure

fig = Figure(width=600, height=400)

m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(geo_json_data).add_to(m)

fig.add_child(m)

## Estilizar

Puedes proporcionar una función de la forma `lambda feature {}` que establece el estilo de cada característica.

In [ ]:
m = folium.Map([43, -100], width='50', height='50', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        "fillColor": "#ffff00",
        "color": "black",
        "weight": 2,
        "dashArray": "10, 10",
    },
).add_to(m)

m

Por ejemplo, si quieres visualizar en verde todos los estados cuyo nombre contenga la letra "E", sólo tienes que hacerlo

In [ ]:
m = folium.Map([43, -100], width='50', height='50', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        "fillColor": "green"
        if "e" in feature["properties"]["name"].lower()
        else "#ffff00",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
    },
).add_to(m)

m

Para poder decir que un mapa es coroplético, tenemos que calcular un color para cada estado.

Imaginemos que queremos plotear un mapa coroplético del desempleo en los Estados Unidos. Primero, vamos a cargar los datos con el desempleo en los EE.UU. de octubre de 2012.

In [ ]:
import pandas as pd

US_Unemployment_Oct2012 = f"{url}/US_Unemployment_Oct2012.csv"
unemployment = pd.read_csv(US_Unemployment_Oct2012)

unemployment.head(5)

Ahora necesitamos crear una función que mapee un valor a un color RGB (de la forma `#RRGGBB`). Para ello, utilizaremos las herramientas de colormap de `folium.colormap`.

Cuando se prefiere tener un conjunto continuo de colores. Esto se puede hacer mediante `Linear`. Por lo que Folium proporciona un (pequeño) conjunto de mapas de colores lineales incorporados, en `folium.colormap.linear`.

Por supuesto, puede necesitar escalar los mapas de colores a sus límites. Esto se puede hacer con `.scale`.

In [ ]:
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    unemployment.Unemployment.min(), unemployment.Unemployment.max()
)

print(colormap(5.0))

colormap

### Documentación: Colormap & colores
- [HTML Color Codes](https://www.rapidtables.com/web/color/html-color-codes.html)
- [Using folium.colormap](https://notebook.community/racu10/emapy/notebooks/Colormaps)
- [branca.colormap](https://python-visualization.github.io/branca/colormap.html)
- [ColorBrewer: Color Advice for Maps](https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3)

También necesitamos convertir la tabla en un diccionario, para asignar una característica a su valor de desempleo.

In [ ]:
unemployment_dict = unemployment.set_index("State")["Unemployment"]

unemployment_dict["AL"]

Ahora podemos crear el mapa coroplético.

In [ ]:
m = folium.Map([43, -100], width='50', height='50', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name="unemployment",
    style_function=lambda feature: {
        "fillColor": colormap(unemployment_dict[feature["id"]]),
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
# Creamos un diccionario con la escala de desempleo
color_dict = {key: colormap(unemployment_dict[key]) for key in unemployment_dict.keys()}

In [ ]:
# Creamos un objeto que incorpora el diccionario anterior
m = folium.Map([43, -100], width='50', height='50', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        "fillColor": color_dict[feature["id"]],
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
).add_to(m)

In [ ]:
# Visualizamos nuestro mapa con una leyenda con la escala de colores del mapa
colormap.caption = "Unemployment color scale"
colormap.add_to(m)

m

## Referencias
- [GeoJSON and choropleth](https://github.com/python-visualization/folium/blob/main/examples/GeoJSON_and_choropleth.ipynb)
- [Folium Documentation](https://python-visualization.github.io/folium/modules.html)